# Decoding from MARBLE and CEBRA embeddings

In this notebook, we compare MARBLE and CEBRA for decoding.

In [ ]:
! pip install cebra statannotations elephant

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator

import cebra

import MARBLE
from rat_utils import *

### Load the data

In [ ]:
!mkdir data
!wget -nc https://dataverse.harvard.edu/api/access/datafile/7609512 -O data/rat_data.pkl

with open('data/rat_data.pkl', 'rb') as handle:
    hippocampus_pos = pickle.load(handle)
    
hippocampus_pos = hippocampus_pos['achilles']

### Define training and test data

In [ ]:
def split_data(data, test_ratio):

    split_idx = int(data['neural'].shape[0] * (1-test_ratio))
    neural_train = data['neural'][:split_idx]
    neural_test = data['neural'][split_idx:]
    label_train = data['continuous_index'][:split_idx]
    label_test = data['continuous_index'][split_idx:]
    
    return neural_train.numpy(), neural_test.numpy(), label_train.numpy(), label_test.numpy()

neural_train, neural_test, label_train, label_test = split_data(hippocampus_pos, 0.2)

In [ ]:
# 40 hz sampling rate (they binned into 25ms epochs

spikes = neural_train[:2000,:].T
spikes = [np.where(spikes[ch,:])[0]/40 for ch in range(120)] 
    
_, ax = plt.subplots(figsize=(8,4))
ax.eventplot(spikes,color='gray')
plt.ylabel('Neurons')
plt.xlabel('Time (s)')

### Preprocess data

In [ ]:
data_train, label_train_marble, pca = convert_spikes_to_rates(neural_train.T, label_train, pca_n=20)
data_test, label_test_marble, _ = convert_spikes_to_rates(neural_test.T, label_test, pca=pca)

## Load the models and get the corresponding embeddings

In [ ]:
#Cebra-time
!wget -nc https://dataverse.harvard.edu/api/access/datafile/7609517 -O data/cebra_time_achilles_32D.pt
cebra_time_model = cebra.CEBRA.load("data/cebra_time_achilles_32D.pt")
cebra_time_train = cebra_time_model.transform(neural_train)
cebra_time_test = cebra_time_model.transform(neural_test)

#Cebra-behaviour
!wget -nc https://dataverse.harvard.edu/api/access/datafile/7609520 -O data/cebra_behaviour_achilles_32D.pt
cebra_behaviour_model = cebra.CEBRA.load("data/cebra_behaviour_achilles_32D.pt")
cebra_behaviour_train = cebra_behaviour_model.transform(neural_train)
cebra_behaviour_test = cebra_behaviour_model.transform(neural_test)

#MARBLE
!wget -nc https://dataverse.harvard.edu/api/access/datafile/7659512 -O data/marble_achilles_32D.pth
marble_model = MARBLE.net(data_train, loadpath="data/marble_achilles_32D.pth")
data_train = marble_model.transform(data_train)
data_test = marble_model.transform(data_test)

## Decode the labels from the embeddings

### Decode the position and direction from the trained hypothesis models

In [ ]:
cebra_time_decode = decoding_pos_dir(cebra_time_train, cebra_time_test, label_train, label_test)
cebra_behaviour_decode = decoding_pos_dir(cebra_behaviour_train, cebra_behaviour_test, label_train, label_test)
marble_decode = decoding_pos_dir(data_train.emb, data_test.emb, label_train_marble, label_test_marble)

### Visualize the decoding results and loss - decoding performance

In [ ]:
results = [cebra_time_decode[4][:-1], cebra_behaviour_decode[4][:-1],  marble_decode[4]]

results = pd.DataFrame(data=np.vstack(results).T,columns=['c-time','c-behaviour', 'marble'])
results = results.melt()
results.columns = ['model','accuracy']

f, ax = plt.subplots(figsize=(4,4))
sns.despine(bottom=True, left=True)

random_sampling = np.random.randint(low=0, high=results.shape[0], size=(200,))

sns.stripplot(
    data=results.iloc[random_sampling,:], x="model", y="accuracy", order=['c-time','c-behaviour', 'marble'], 
    dodge=True, alpha=.5, zorder=1, color='gray',
)

sns.pointplot(
    data=results, x="model", y="accuracy",
    join=False, dodge=.8 - .8 / 3, palette="dark",
    markers="d", scale=.75, errorbar=None
)

pairs=[("c-time", "marble"), ("c-behaviour", "marble")]

annotator = Annotator(ax, pairs, data=results, x="model", y="accuracy",)
annotator.configure(test='Wilcoxon', text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.tight_layout()

In [ ]:
from scipy.stats import sem

fig = plt.figure(figsize=(10,4))

ax1= plt.subplot(121)
ax1.bar(np.arange(3), 
        [cebra_time_decode[1], cebra_behaviour_decode[1], marble_decode[1]], 
        width = 0.5, color = 'gray')

ax1.errorbar(np.arange(3),  
             [cebra_time_decode[1], cebra_behaviour_decode[1], marble_decode[1]],
             yerr=[sem(cebra_time_decode[4]), sem(cebra_behaviour_decode[4]), sem(marble_decode[4])], 
             fmt='none', ecolor='black', capsize=5)


ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xticks(np.arange(3))
ax1.set_xticklabels(['time','behaviour','marble'])
ax1.set_ylabel('Median err. [m]')

plt.legend(bbox_to_anchor=(1,1), frameon = False )
plt.show()

In [ ]:
fig = plt.figure(figsize=(5,6))

t = label_test_marble[:,0].shape[0]

ax1= plt.subplot(311)
ax1.plot(np.arange(t)/40, label_test_marble[:,0])
ax1.plot(np.arange(t)/40, marble_decode[3][:,0])
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Position [m]')
ax1.set_title('MARBLE')

ax2= plt.subplot(312)
ax2.plot(np.arange(t)/40, label_test_marble[:,0])
ax2.plot(np.arange(t+1)/40, cebra_time_decode[3][:,0])
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Position [m]')
ax2.set_title('CEBRA (time)')

ax3 = plt.subplot(313)
ax3.plot(np.arange(t)/40, label_test_marble[:,0])
ax3.plot(np.arange(t+1)/40, cebra_behaviour_decode[3][:,0])
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Position [m]')
ax3.set_title('CEBRA (position)')

plt.tight_layout()